# Importar librerías



In [ ]:
#Conjunto de datos
from sklearn.datasets import load_iris
#Árbol de decisión
from sklearn import tree
#Análisis
import pandas as pd
#Segmentación de datos (entrenamiento, validación y prueba)
from sklearn.model_selection import train_test_split
#Vizualización
import seaborn as sns #Box plot
import matplotlib.pyplot as plt #Distribución
#Importamos la función confusion_matrix desde sklearn.metrics
from sklearn.metrics import confusion_matrix
#Libreria para metricas
from sklearn.metrics import accuracy_score

# Carga de conjunto de datos

In [ ]:
#Dataset
iris = load_iris()
#Dataframe
df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
#Ver información de una dataset
df.info()
#Etiqueta de salida (setosa, versicolor y virginica)
df['target'] = iris.target
#Adición de nombres a etiquetas de salida
df['species'] = df['target'].map(dict(enumerate(iris.target_names)))

In [ ]:
# Mostrar las primeras filas del DataFrame completo
print(df.head(10))

# Estadística datos

In [ ]:
# Estadísticas descriptivas
print(df.describe())

# Vizualización de distribución

In [ ]:
# Establece el tamaño de la figura (ancho x alto en pulgadas)
plt.figure(figsize=(8, 6))

# Crea un gráfico de dispersión con seaborn
# - data: el DataFrame que contiene los datos
# - x: columna para el eje X ('largo del sépalo')
# - y: columna para el eje Y ('ancho del sépalo')
# - hue: color según la especie (distingue clases con diferentes colores)
# - palette: paleta de colores ('Set2' es una paleta pastel atractiva)
# - s: tamaño de los puntos en el gráfico
sns.scatterplot(data=df,
                x='sepal length (cm)',
                y='sepal width (cm)',
                hue='species',
                palette='Set2',
                s=100)

# Título del gráfico
plt.title("Gráfico de Dispersión: Largo vs Ancho del Sépalo")
# Etiqueta del eje X
plt.xlabel("Largo del Sépalo (cm)")
# Etiqueta del eje Y
plt.ylabel("Ancho del Sépalo (cm)")
# Activa la grilla para facilitar la lectura del gráfico
plt.grid(True)
# Muestra el gráfico en pantalla
plt.show()


# Vizualización Box plot

In [ ]:
# Creamos una figura general con un tamaño de 14x10 pulgadas
plt.figure(figsize=(14, 10))

# Recorremos cada característica numérica del conjunto de datos 'iris'
for i, column in enumerate(iris.feature_names):

    # Creamos una subgráfica en una cuadrícula de 2 filas x 2 columnas
    # 'i+1' indica la posición actual (1 a 4)
    plt.subplot(2, 2, i+1)

    # Dibujamos un boxplot con seaborn
    # En el eje X colocamos las especies (setosa, versicolor, virginica)
    # En el eje Y colocamos la característica numérica actual (ej. sepal length)
    # 'data=df' indica que los datos vienen del DataFrame 'df'
    sns.boxplot(x='species', y=column, data=df)

    # Añadimos un título a cada subgráfico con el nombre de la característica
    plt.title(f'Distribución de {column}')

# Ajustamos el diseño para que los subgráficos no se superpongan
plt.tight_layout()

# Mostramos todos los gráficos en pantalla
plt.show()


# Segmentación de datos 70%, 15%, 15%

In [ ]:
# División inicial en 70% entrenamiento y 30% temporal
X = df[iris.feature_names]
y = df['target']
#Datos de entrenamiento 70%
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.30, random_state=42, stratify=y)

# División del 30% restante en validación y prueba (50/50 => 15% cada uno)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.50, random_state=42, stratify=y_temp)

print(f"Entrenamiento: {len(X_train)}, Validación: {len(X_val)}, Prueba: {len(X_test)}")


# Carga de modelo y entrenamiento

In [ ]:
# -------------------------------
#     🔁 DEFINICIÓN DEL MODELO
# -------------------------------

# Creamos un modelo de árbol de decisión para clasificación
# El parámetro 'random_state=42' se usa para fijar un generador
# de valores aleatorios

modelo = tree.DecisionTreeClassifier(random_state=42)

# -------------------------------
#         🔧 ENTRENAMIENTO
# -------------------------------

# Entrenamos (ajustamos) el modelo usando los datos de entrenamiento 70%
# X_train: conjunto de variables independientes (características)
# y_train: conjunto de variables dependientes (clase o etiqueta)
modelo.fit(X_train, y_train)

In [ ]:
# ------------------------------------
#     🌳 VISUALIZACIÓN DEL ÁRBOL
# ------------------------------------

# Usamos la función plot_tree de sklearn para dibujar el árbol de decisión entrenado
# Esto muestra gráficamente cómo el modelo toma decisiones (las divisiones en los nodos)
# Es útil para interpretar el modelo y entender qué atributos está utilizando
tree.plot_tree(modelo)


In [ ]:
# ---------------------------------------------
#   🔮 PREDICCIÓN SOBRE LOS DATOS DE VALIDACIÓN
# ---------------------------------------------

# Utilizamos el modelo ya entrenado para hacer predicciones sobre el conjunto de validación (X_val)
# Esto permite evaluar el rendimiento del modelo en datos que no se usaron en el entrenamiento
# y_pred_val almacenará las clases predichas por el modelo para X_val
y_pred_val = modelo.predict(X_val)

In [ ]:
# ------------------------------------
# 📏 MÉTRICAS DE EVALUACIÓN DEL MODELO
# ------------------------------------
# 🔹 Accuracy: porcentaje total de aciertos
# 2. Accuracy de validacion
accuracy_v = accuracy_score(y_val, y_pred_val)
# Imprimimos las métricas %
print("Accuracy:", accuracy_v * 100)

In [ ]:
# ---------------------------------------
#     🔮 PREDICCIÓN CON EL MODELO
# ---------------------------------------

# Usamos el modelo ya entrenado para hacer predicciones sobre los datos de prueba (X_test)
# El resultado (y_pred) será un array con las clases predichas para cada observación de prueba
y_pred = modelo.predict(X_test)

In [ ]:
# ------------------------------------
# 📏 MÉTRICAS DE EVALUACIÓN DEL MODELO
# ------------------------------------
# 1. Accuracy de prueba
# 🔹 Accuracy: porcentaje total de aciertos
accuracy_p = accuracy_score(y_test, y_pred)
# Imprimimos las métricas %
print("Accuracy:", accuracy_p * 100)


# Vizualización de información

In [ ]:
# ------------------------------------
#       📊 MATRIZ DE CONFUSIÓN
# ------------------------------------

# Calculamos la matriz de confusión comparando los valores reales (y_test)
# con las predicciones hechas por el modelo (y_pred)
# La matriz indica cuántas predicciones fueron correctas y cuántas se equivocaron
cm = confusion_matrix(y_test, y_pred)

# --------------------------------------------
#      📈 GRÁFICO DE LA MATRIZ DE CONFUSIÓN
# --------------------------------------------

# Usamos seaborn para crear un mapa de calor (heatmap) de la matriz
# 'annot=True' muestra los números en cada celda
# 'fmt="d"' indica que los valores son enteros
# 'cmap="Blues"' da el color azul al gráfico
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")

# Etiquetas de los ejes
plt.xlabel("Predicción")    # Eje X: lo que predijo el modelo
plt.ylabel("Verdadero")     # Eje Y: las clases reales
plt.title("Matriz de Confusión")  # Título del gráfico

# Muestra el gráfico en pantalla
plt.show()

#Exportar modelo ML

In [ ]:
#Exportar modelo de machine learning
import joblib # joblib sirve para serializar eficientemente objetos Python,
               # en especial modelos de machine learning, permitiendo guardarlos
               # y cargarlos más rápido, lo cual es útil para la persistencia.

joblib.dump(modelo, 'modelo_iris.pkl')
# Esta línea utiliza la función 'dump' de joblib para guardar (serializar) el
# objeto Python llamado 'modelo' (que se asume es un modelo de Machine Learning
# previamente entrenado, como un clasificador de Scikit-learn) en un archivo
# binario llamado 'modelo_iris.pkl'. Esto permite reutilizar el modelo sin tener
# que entrenarlo de nuevo.


#Cargar modelo ML


In [ ]:
# Nombre del archivo donde se guardó el modelo
nombre_archivo = 'modelo_iris.pkl'

# Cargar el modelo desde el archivo binario
# El modelo cargado se asigna a la variable 'modelo_cargado'
modelo_cargado = joblib.load(nombre_archivo)

print(f"✅ Modelo cargado exitosamente desde '{nombre_archivo}'.")
